In [1]:
import numpy as np
import pandas as pd
import string 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#systeme
import sys
sys.setrecursionlimit(200000)

#nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams

#import warnings; warnings.simplefilter('ignore')

import re

#import spacy
#from spacy_lefff import lefffLemmatizer

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

import fr_core_news_sm
nlp = fr_core_news_sm.load()

#%load_ext Cython

#wolf
#from frenetic.packet import *
#fwn = FreNetic("C:/Users/Utilisateur/Desktop/simplon/projetA1/wolf.xml") 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


C:\Users\Dell\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


<h1 align='center'> Traitement du langage naturel </h1>
<h2 align='center'> Analyse lexicale de sentiment de tweets</h2>
<br>
<br>
<hr>
<h2 align='center'><strong>1ere Partie: </strong>Pre-processing</h2>

### 1/ Import du csv

In [2]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [3]:
tweets = pd.read_csv("./french-tweets-politique/tweets_unique.csv")

In [4]:
tweets.head()

,Unnamed: 0,date,text,hashtags
0,0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NaN
1,1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NaN
2,2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NaN
3,3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NaN
4,4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"['Macron', 'Bellatar', 'ConseilDesVilles']"


In [5]:
tweets.rename(columns={'Unnamed: 0':'num'}, inplace=True)

In [6]:
tweets.head()

,num,date,text,hashtags
0,0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NaN
1,1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NaN
2,2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NaN
3,3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NaN
4,4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"['Macron', 'Bellatar', 'ConseilDesVilles']"


In [7]:
tw = tweets.copy()

### 2/ Normalisation du texte

In [8]:
low = lambda x: str(x).lower()
tweets['text'] = tweets.text.apply(low)
tweets.head()

,num,date,text,hashtags
0,0,Sun Mar 18 13:45:13 +0000 2018,au lieu d'expulser des diplomates russes il eu...,NaN
1,1,Sun Mar 18 15:47:19 +0000 2018,"selon l’ex ambassadeur anglais en ouzbékistan,...",NaN
2,2,Sun Mar 18 11:48:32 +0000 2018,@philipj37 @eugeniebastie tellement vrai.\r\ns...,NaN
3,3,Sun Mar 18 15:41:24 +0000 2018,"bombe: le capitaine barril, ancien commandant ...",NaN
4,4,Sun Mar 18 14:32:39 +0000 2018,dans la même semaine #macron nomme le très amb...,"['Macron', 'Bellatar', 'ConseilDesVilles']"


### 3/ Supression des caracteres speciaux et double espaces

In [9]:
tweets['text'] = tweets['text'].apply(lambda x: re.sub('[!@#$:’).";,?&%=>(<''©-]', '', x.lower()))
tweets['text'] = tweets['text'].apply(lambda x: re.sub('  ', ' ', x))
tweets['text'].head()

0    au lieu d'expulser des diplomates russes il eu...
1    selon lex ambassadeur anglais en ouzbékistan l...
2    philipj37 eugeniebastie tellement vrai\r\ns'oc...
3    bombe le capitaine barril ancien commandant du...
4    dans la même semaine macron nomme le très ambi...
Name: text, dtype: object

In [10]:
for i in range(len(tweets['text'])):
    tweets['text'][i] = " ".join([word for word in tweets['text'][i].split()
                                if 'http' not in word and '@' not in word and '<' not in word])
tweets['text'].head()

0    au lieu d'expulser des diplomates russes il eu...
1    selon lex ambassadeur anglais en ouzbékistan l...
2    philipj37 eugeniebastie tellement vrai s'occup...
3    bombe le capitaine barril ancien commandant du...
4    dans la même semaine macron nomme le très ambi...
Name: text, dtype: object

### 4/ Tokénisation

In [11]:
tweets.dtypes

num         object
date        object
text        object
hashtags    object
dtype: object

In [12]:
tweets['text'].astype('str', inplace=True).head()

0    au lieu d'expulser des diplomates russes il eu...
1    selon lex ambassadeur anglais en ouzbékistan l...
2    philipj37 eugeniebastie tellement vrai s'occup...
3    bombe le capitaine barril ancien commandant du...
4    dans la même semaine macron nomme le très ambi...
Name: text, dtype: object

In [13]:
clean = lambda tx: word_tokenize(tx)
tweets.text = tweets.text.apply(clean)
tweets.text.head()

0    [au, lieu, d'expulser, des, diplomates, russes...
1    [selon, lex, ambassadeur, anglais, en, ouzbéki...
2    [philipj37, eugeniebastie, tellement, vrai, s'...
3    [bombe, le, capitaine, barril, ancien, command...
4    [dans, la, même, semaine, macron, nomme, le, t...
Name: text, dtype: object

### 5/ Stop-Words

In [14]:
stopW = stopwords.words('french')
#print(stopW)
filtered_sentence = [w for w in tweets['text'] if not w in stopW] 
# remove URLs, RTs, and twitter handles
for i in range(len(filtered_sentence)):
    filtered_sentence[i] =" ".join([word for word in filtered_sentence[i]
                               if 'http' not in word  and '<' not in word])
filtered_sentence[0]

"au lieu d'expulser des diplomates russes il eut été plus concrets et plus pragmatiques de s'occuper des ressortissants info pakistanais présumés responsables de milliers de viols d'enfants à telford mme may a opté pour la facilité face à une monstruosité ethnique"

In [15]:
def filtration(tk: list ):#texte tokenisé sous format de liste
    l_filtree = []
    for word in tk:
        if word not in stopW:
            l_filtree.append(word)
    return l_filtree

In [16]:
filtered_sentence = tweets['text'].apply(filtration)
filtered_sentence.head()

0    [lieu, d'expulser, diplomates, russes, plus, c...
1    [selon, lex, ambassadeur, anglais, ouzbékistan...
2    [philipj37, eugeniebastie, tellement, vrai, s'...
3    [bombe, capitaine, barril, ancien, commandant,...
4    [semaine, macron, nomme, très, ambigu, l'islam...
Name: text, dtype: object


### 6/ Lemmatisation

In [17]:
#fonction de lemmatisation
def lemma(l:list):
    new_l = ''
    for word in l:
        new_l += ' '+word
    # strip retire les espaces au debut et a la fin de la chaine de caractere
    new_l = new_l.strip()
    n = nlp(new_l)
    # boucle sur n et applique la fonction lemma_ sur chacun des tokens
    x = []
    for z in n:
        x.append(z.lemma_)
    return x

In [18]:
tweetslemma = filtered_sentence.apply(lemma)

In [19]:
tweetslemma.head()

0    [lieu, de, expulser, diplomate, russe, plus, c...
1    [selon, lex, ambassadeur, anglais, ouzbékistan...
2    [philipj37, eugeniebastie, tellement, vrai, si...
3    [bombe, capitaine, barril, ancien, commander, ...
4    [semaine, macron, nommer, très, ambigu, le, is...
Name: text, dtype: object

In [20]:
tweetslemma = tweetslemma.apply(filtration)

<br>
<br>
<hr>
<h2 align='center'><b>2eme Partie: </b>Attribution d'un taux de polarite de sentiment</h2>

### 1/ Agregation des donnees

In [21]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [22]:
pol = pd.read_csv("./mots.csv", sep=",", encoding = "ISO-8859-1", error_bad_lines=False)

In [23]:
#Transformer le df pol en deux listes
pP = pol[pol['polarity']=='positive'].word.values
pN = pol[pol['polarity']=='negative'].word.values

In [24]:
pN = [w.strip() for w in pN]

### 2/ Fonction de calcul des polarite

In [25]:
def calculpolarite(text: str):
    scoreP = [True for word in text if word in pP].count(True)
    scoreN = [True for word in text if word in pN].count(True)
    if(scoreP>scoreN):
        p = 'positif'
        s = "{0:.2f}".format(scoreP/(scoreN+scoreP))
    elif(scoreP==scoreN):
        p= 'neutre'
        s=0
    else:
        p= 'negatif' 
        s ="{0:.2f}".format(scoreN/(scoreN+scoreP))
    return [p,s]

In [26]:
# test pour nouvelle fonction
rs =list(tweetslemma[:5].apply(calculpolarite))

In [27]:
# Suite test
rs = np.array(rs)
rs[:,1]

array(['0.83', '1.00', '1.00', '0', '1.00'], dtype='<U7')

In [28]:
polarite = list(tweetslemma.apply(calculpolarite))
polarite = np.array(polarite)

In [29]:
tweets['sentiment'] = polarite[:,0]
tweets['taux']= polarite[:,1]
tweets.head()

,num,date,text,hashtags,sentiment,taux
0,0,Sun Mar 18 13:45:13 +0000 2018,"[au, lieu, d'expulser, des, diplomates, russes...",NaN,positif,0.83
1,1,Sun Mar 18 15:47:19 +0000 2018,"[selon, lex, ambassadeur, anglais, en, ouzbéki...",NaN,positif,1.00
2,2,Sun Mar 18 11:48:32 +0000 2018,"[philipj37, eugeniebastie, tellement, vrai, s'...",NaN,positif,1.00
3,3,Sun Mar 18 15:41:24 +0000 2018,"[bombe, le, capitaine, barril, ancien, command...",NaN,neutre,0
4,4,Sun Mar 18 14:32:39 +0000 2018,"[dans, la, même, semaine, macron, nomme, le, t...","['Macron', 'Bellatar', 'ConseilDesVilles']",positif,1.00


In [30]:
# from collections import Counter

# def evaltweet(s:list):
#     pola = [' '.join(pol[pol.word == e].polarity) for e in s]
#     pola = Counter(pola)
#     polap = pola.get('positive', 0)
#     polan = pola.get('negative', 0)
#     return '¯\\_(ツ)_/¯' if (polap + polan) == 0 else {'positif':polap / (polap + polan), 'negatif':polan / (polap + polan)}

In [31]:
# def evaltweet2(s:list):
#     pola = [' '.join(pol[pol.word == e].polarity) for e in s]
#     pola = Counter(pola)
#     polap = pola.get('positive', 0)
#     polan = pola.get('negative', 0)
#     if (polap == polan):
#         p = 'neutre'
#     elif (polap>polan):
#         p= 'positif'
#     else:
#         p= 'negatif'
#     return p

In [32]:
# def help_apply():
#     # de 0 a 7328, boucler tous les 1000
#     deb = 0
#     fin = 500
#     polarite = []
#     for i in range(16):
#         polarite.append(tweetslemma[deb:fin].apply(evaltweet2))
#         deb += 500
#         fin += 500
#         if(fin>7328):
#             fin = 7328
#     return polarite

In [33]:
# tweets['processed_text'] = tweetslemma
# tweets.head()

In [34]:
tweetslemma.shape

(7327,)

## Preparation des donnees pour l'export

In [35]:
# for i in pol.word:
#     if (tweets.text[0].find(i)) != -1:
#         print("Ce tweet contient [" + i + "] avec la polarité: " + pol.polarity)

In [36]:
tweets.hashtags.replace("[][']", '', regex=True, inplace=True)

In [37]:
textjoin = lambda l : ','.join([w for w in l])
t = tweetslemma.apply(textjoin)
tweets['words'] = t

In [38]:
tweets.head()

,num,date,text,hashtags,sentiment,taux,words
0,0,Sun Mar 18 13:45:13 +0000 2018,"[au, lieu, d'expulser, des, diplomates, russes...",NaN,positif,0.83,"lieu,expulser,diplomate,russe,plus,concret,plu..."
1,1,Sun Mar 18 15:47:19 +0000 2018,"[selon, lex, ambassadeur, anglais, en, ouzbéki...",NaN,positif,1.00,"selon,lex,ambassadeur,anglais,ouzbékistan,``,n..."
2,2,Sun Mar 18 11:48:32 +0000 2018,"[philipj37, eugeniebastie, tellement, vrai, s'...",NaN,positif,1.00,"philipj37,eugeniebastie,tellement,vrai,si,occu..."
3,3,Sun Mar 18 15:41:24 +0000 2018,"[bombe, le, capitaine, barril, ancien, command...",NaN,neutre,0,"bombe,capitaine,barril,ancien,commander,gign,«..."
4,4,Sun Mar 18 14:32:39 +0000 2018,"[dans, la, même, semaine, macron, nomme, le, t...","Macron, Bellatar, ConseilDesVilles",positif,1.00,"semaine,macron,nommer,très,ambigu,islamisme,be..."


In [39]:
#Recuperer la colonne text du df de depart
tweets['text'] = tw.text
tweets.head()

,num,date,text,hashtags,sentiment,taux,words
0,0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NaN,positif,0.83,"lieu,expulser,diplomate,russe,plus,concret,plu..."
1,1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NaN,positif,1.00,"selon,lex,ambassadeur,anglais,ouzbékistan,``,n..."
2,2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NaN,positif,1.00,"philipj37,eugeniebastie,tellement,vrai,si,occu..."
3,3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NaN,neutre,0,"bombe,capitaine,barril,ancien,commander,gign,«..."
4,4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"Macron, Bellatar, ConseilDesVilles",positif,1.00,"semaine,macron,nommer,très,ambigu,islamisme,be..."


In [40]:
#Remplace les nan par NO HASHTAG
tweets.hashtags.replace(np.nan, 'NO HASHTAG' ,inplace=True)
tweets.head()

,num,date,text,hashtags,sentiment,taux,words
0,0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NO HASHTAG,positif,0.83,"lieu,expulser,diplomate,russe,plus,concret,plu..."
1,1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NO HASHTAG,positif,1.00,"selon,lex,ambassadeur,anglais,ouzbékistan,``,n..."
2,2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NO HASHTAG,positif,1.00,"philipj37,eugeniebastie,tellement,vrai,si,occu..."
3,3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NO HASHTAG,neutre,0,"bombe,capitaine,barril,ancien,commander,gign,«..."
4,4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"Macron, Bellatar, ConseilDesVilles",positif,1.00,"semaine,macron,nommer,très,ambigu,islamisme,be..."


In [41]:
# Export du df en CSV
tweets.to_csv("tweetsAnalyse.csv", index=False, encoding='utf-8')

## Definition de la fonction de determination de polarite a partir d'un texte

In [42]:
# The function polarity_feeling takes a string as entry and returns the text's feeling polarity: 'neutre', 'positif' or 'negatif'
# and the rate in a list. 
def polarity_feeling(text: str):
    # text to lower case 
    # delete punctuation
    text =re.sub('[!@#$:’).";,?&%=>(<''©-]', '', str(text).lower())
    # delete doubble whitespace
    text = re.sub('  ', ' ', text)
    # delete @, > and 'http'
    text = " ".join([word for word in text.split()
                                if 'http' not in word and '@' not in word and '<' not in word])
    # Tokenisation
    text = word_tokenize(text)
    # Lemmatization
    text = lemma(text)
    # delete stopwords
    text = filtration(text)
    # define the feeling polarity 
    return calculpolarite(text)
    
    

In [43]:
polarity_feeling('Bonjour comment vas tu petit chat?')

['neutre', 0]

In [44]:
tw.head()

,num,date,text,hashtags
0,0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NaN
1,1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NaN
2,2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NaN
3,3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NaN
4,4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"['Macron', 'Bellatar', 'ConseilDesVilles']"


In [45]:
_pol = tw[:5].text.apply(polarity_feeling)

In [46]:
_pol

0    [positif, 0.83]
1    [positif, 1.00]
2    [positif, 1.00]
3        [neutre, 0]
4    [positif, 1.00]
Name: text, dtype: object